In [11]:
from util.sql_switch import get_query_native

df = get_query_native(
    "SELECT dt.id as tweet_id, text, dt.conversation_id, dt.author_id, dt.twitter_id, dt.tn_parent_id, sentiment,sentiment_value,created_at, is_toxic, toxic_value, centrality, baseline_vision, n_posts, is_root_author FROM delab_tweet dt join delab_conversationauthormetrics cam on dt.conversation_id = cam.conversation_id and dt.tw_author_id = cam.author_id")

df.head(3)
# df.describe()

using postgres


/home/dehne/PycharmProjects/delab/venv/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,tweet_id,text,conversation_id,author_id,twitter_id,tn_parent_id,sentiment,sentiment_value,created_at,is_toxic,toxic_value,centrality,baseline_vision,n_posts,is_root_author
0,33646,"@SabinaMietzer Oder weil man befürchtet Ziel von cancel culture zu werden, weil man dem aktuellen Mainstream entgegensteht.\n\nEs gibt viele Gründe anonym bleiben zu wollen. Wichtig ist, diese Freiheit haben zu dürfen.",1518867661509300225,234847109,1518959424500645888,1.518868e+18,negative,-0.2500,2022-04-26 14:25:22+00:00,False,0.000000,0.0,0.09375,1,False
1,44325,@tomdabassman @Minnimina #DasIstNichtMeinKrieg,1523205712632975360,1506535819238166534,1523210102819414018,1.523206e+18,None,NaN,2022-05-08 07:56:03+00:00,False,0.000000,0.0,0.09375,1,False
2,28879,@ManuelaSchwesig @AnkeRehlinger 4 starke kompetente attraktive SPD-Frauen an der Spitze ihrer Länder quer durch die Bundesrepublik! Bin großer Fan!!!Toll 😍😍😍😍,1518647025839001601,1443494838616403972,1518650024443453441,1.518647e+18,negative,-0.7243,2022-04-25 17:55:55+00:00,False,0.090135,0.0,0.09375,1,False


1. Isolate authors that have a high centrality or baseline vision
2. Compare the selection
3. Compute longest flows per conversation (in the creation script)
4. Filter longest flows whether central authors are present (in the creation script)
5. Return the longest flows (in the creation script)

In [12]:
import pandas as pd
# 1. Isolate authors that have a really high centrality / baseline vision

import pandas as pd
from sklearn import preprocessing

df_central_authors = df.drop(
    ["sentiment_value", "toxic_value", "sentiment", "is_root_author", "is_toxic", "text", "created_at", "twitter_id", "tn_parent_id"], axis=1)
df_repetition_probs = df.drop(
    ["sentiment_value", "toxic_value", "sentiment", "is_root_author", "is_toxic", "text", "created_at", "centrality",
     "baseline_vision", "n_posts", "author_id", "twitter_id", "tn_parent_id"], axis=1)
df_repetition_probs


,tweet_id,conversation_id
0,33646,1518867661509300225
1,44325,1523205712632975360
2,28879,1518647025839001601
3,33729,1518867661509300225
4,44426,1522899712055074816
...,...,...
80450,1359,1504619833773772805
80451,1342,1504619833773772805
80452,1337,1504619833773772805
80453,1349,1504619833773772805


In [13]:
df_repetition_probs = df_repetition_probs.groupby(["conversation_id"]).count()
grouped_ca = df_central_authors.groupby(["conversation_id", "author_id"]).mean()
grouped_ca = grouped_ca.drop("tweet_id", axis=1)
grouped_ca = grouped_ca.join(df_repetition_probs)
grouped_ca = grouped_ca.assign(repetition_prob=grouped_ca.n_posts / grouped_ca.tweet_id)
grouped_ca = grouped_ca.drop("tweet_id", axis=1)
grouped_ca = grouped_ca.apply(lambda x: x / x.max())
grouped_ca['mean'] = grouped_ca.mean(axis=1)
mean_largest = grouped_ca.nlargest(20, "mean")
# this is the result list, first element of tuple is conversation, second the author
mean_largest.index.tolist()


[(1508899545060720644, 9204502),
 (1512323466652958723, 322675289),
 (1509478496208306178, 18047862),
 (1520667401095888901, 359982860),
 (1511987996135940097, 481428317),
 (1518625103679995905, 208555813),
 (1518662374877655040, 1331650559518990336),
 (1511895403620450306, 342863309),
 (1525010538983464961, 1298381117519589377),
 (1509496191515611136, 1163922594569101315),
 (1512326919009673220, 1314975287294083073),
 (1512187211395047428, 30354991),
 (1542127567297290245, 588686739),
 (1543200394477805568, 126585908),
 (1508884357242621953, 2522911828),
 (1509412070940368900, 1168167671151628290),
 (1510890678435336193, 280681667),
 (1512054183721058322, 1432335316795465728),
 (1519496208288403456, 42558679),
 (1521571487785881601, 1114675538)]

In [15]:
df.style.set_properties(**{'text-align': 'left'})

example_conversation_id = mean_largest.index.tolist()[0][0]
print(example_conversation_id)
example = df[df["conversation_id"] == example_conversation_id]
example.author_id



1508899545060720644


101      1357678561281404930
182      1450353392501628930
295      1453927879201460237
332      1420802389666304011
576      1241651473315958784
                ...         
80042              124169534
80094    1268526247157600259
80095    1490386676920852484
80098    1493533218892652545
80099             2472048869
Name: author_id, Length: 766, dtype: int64